# Tutorial 2: Forecast after DA


**Goal**: To run a cycled data assimilation experiment.
[`cycled_exp.py`](https://github.com/lkugler/DART-WRF/blob/master/generate_free.py) contains an example which will be explained here:

Now there are two options:
1) To start a forecast from an existing forecast, i.e. from WRF restart files
2) To start a forecast from defined thermodynamic profiles, i.e. from a `wrf_profile`


### Restart a forecast
To run a forecast from initial conditions of a previous forecasts, we import these modules
```python
import datetime as dt
from dartwrf.workflows import WorkFlows
```

Let's say you want to run a forecast starting at 9 UTC until 12 UTC.
Initial conditions shall be taken from a previous experiment in `/user/test/data/sim_archive/exp_abc` which was initialized at 6 UTC and there are WRF restart files for 9 UTC.
Then the code would be

```python
prior_path_exp = '/user/test/data/sim_archive/exp_abc'
prior_init_time = dt.datetime(2008,7,30,6)
prior_valid_time = dt.datetime(2008,7,30,9)

w = WorkFlows(exp_config='cfg.py', server_config='srvx1.py')

begin = dt.datetime(2008, 7, 30, 9)
end = dt.datetime(2008, 7, 30, 12)

w.prepare_WRFrundir(begin)

w.prepare_IC_from_prior(prior_path_exp, prior_init_time, prior_valid_time)

w.run_ENS(begin=begin,  # start integration from here
         end=end,      # integrate until here
         output_restart_interval=9999,  # do not write WRF restart files
         )
```
Note that we use predefined workflow functions like `run_ENS`.


### Forecast run after Data Assimilation
In order to continue after assimilation you need the posterior = prior (1) + increments (2)

1. Set posterior = prior
```python
id = w.prepare_IC_from_prior(prior_path_exp, prior_init_time, prior_valid_time, depends_on=id)
```

2) Update posterior with increments from assimilation
After this, the wrfrst files are updated with assimilation increments from DART output and copied to the WRF's run directories so you can continue to run the forecast ensemble.
```python
id = w.update_IC_from_DA(time, depends_on=id)
```

3) Define how long you want to run the forecast and when you want WRF restart files. Since they take a lot of space, we want as few as possible.

```python
timedelta_integrate = dt.timedelta(hours=5)
output_restart_interval = 9999  # any value larger than the forecast duration
```

If you run DA in cycles of 15 minutes, it will be
```python
timedelta_integrate = dt.timedelta(hours=5)
timedelta_btw_assim = dt.timedelta(minutes=15)
output_restart_interval = timedelta_btw_assim.total_seconds()/60
```


3) Run WRF ensemble
```python
id = w.run_ENS(begin=time,  # start integration from here
               end=time + timedelta_integrate,  # integrate until here
               output_restart_interval=output_restart_interval,
               depends_on=id)
```
